In [4]:
###############################################################################
# The Institute for the Design of Advanced Energy Systems Integrated Platform
# Framework (IDAES IP) was produced under the DOE Institute for the
# Design of Advanced Energy Systems (IDAES).
#
# Copyright (c) 2018-2023 by the software owners: The Regents of the
# University of California, through Lawrence Berkeley National Laboratory,
# National Technology & Engineering Solutions of Sandia, LLC, Carnegie Mellon
# University, West Virginia University Research Corporation, et al.
# All rights reserved.  Please see the files COPYRIGHT.md and LICENSE.md
# for full copyright and license information.
###############################################################################

# ML/AI Best Practices: "Selecting Surrogate Model Form/Size for Optimization"
Maintainer: Brandon Paul  
Author: Brandon Paul  
Updated: 2023-06-01  

In this notebook we demonstrate the use of model and solver statistics to select the best surrogate model. For this purpose we trained (offline) different models with ALAMO, PySMO for three basis forms, and TensorFlow Keras. The surrogates are imported into the notebook, and the IDAES flowsheet is constructed and solved.

## 1. Introduction

This example demonstrates autothermal reformer optimization leveraging the ALAMO, PySMO and Keras surrogate trainers, and compares key indicators of model performance. In this notebook, IPOPT will be run with statistics using ALAMO, PySMO Polynomial, PySMO RBF, PySMO Kriging and Keras surrogate models to assess each model type for flowsheet integration and tractability.

## 2. Problem Statement 

Within the context of a larger Natural Gas Fuel Cell (NGFC) system, the autothermal reformer unit is used to generate syngas from air, steam, and natural gas. Two input variables are considered for this example (reformer bypass fraction and fuel to steam ratio). The reformer bypass fraction (also called internal reformation percentage) plays an important role in the syngas final composition and it is typically controlled in this process. The fuel to steam ratio is an important variable that affects the final syngas reaction and heat duty required by the reactor.  The syngas is then used as fuel by a solid-oxide fuel cell (SOFC) to generate electricity and heat. 

The autothermal reformer is typically modeled using the IDAES Gibbs reactor and this reactor is robust once it is initialized; however, the overall model robustness is affected due to several components present in the reaction, scaling issues for the largrangean multipliers, and Gibbs free energy minimization formulation. Substituting rigorously trained and validated surrogates in lieu of rigorous unit model equations increases the robustness of the problem.

### 2.1. Inputs: 
- Bypass fraction (dimensionless) - split fraction of natural gas to bypass AR unit and feed directly to the power island
- NG-Steam Ratio (dimensionless) - proportion of natural relative to steam fed into AR unit operation

### 2.2. Outputs:
- Steam flowrate (kg/s) - inlet steam fed to AR unit
- Reformer duty (kW) - required energy input to AR unit
- Composition (dimensionless) - outlet mole fractions of components (Ar, C2H6, C3H8, C4H10, CH4, CO, CO2, H2, H2O, N2, O2)

In [5]:
from IPython.display import Image
from pathlib import Path

## 3. Training Surrogates

Previous Jupyter Notebooks demonstrated the workflow to import data, train surrogate models using [ALAMO](alamo/alamo_flowsheet_optimization_src_test.ipynb), [PySMO](pysmo/pysmo_flowsheet_optimization_src_test.ipynb) and Keras, and develop IDAES's validation plots. To keep this notebook simple, this notebook simply loads the surrogate models trained off line.

Note that the training/loading method includes a "retrain" argument in case the user wants to retrain all surrogate models. Since the retrain method runs ALAMO, PySMO (Polynomial, Radial Basis Functions, and Kriging basis types) and Keras, it takes about an 1 hr to complete the training for all models.

Each run will overwrite the serialized JSON files for previously trained surrogates if retraining is enforced. To retrain individual surrogates, simply delete the desired JSON before running this notebook (for Keras, delete the folder `keras_surrogate/`)

In [6]:
from idaes_examples.mod.surrogates.AR_training_methods import (
    train_load_surrogates,
    SurrType,
)

trained_surr = train_load_surrogates(retrain=False)
# setting retrain to True will take ~ 1 hour to run, best to load if possible
# setting retrain to False will only generate missing surrogates (only if JSON/folder doesn't exist)
# this method trains surrogates and serializes to JSON
# The return value is a set of surrogate types (instances of SurrType) that were trained

# imports to capture long output
from io import StringIO
import sys

'_BlockData'. The class '_BlockData' has been renamed to 'BlockData'.
(deprecated in 6.7.2) (called from d:\anaconda\envs\my-idaes-env\lib\site-
packages\omlt\block.py:33)
Loading existing surrogate models and training missing models.
Any training output will print below; otherwise, models will be loaded without any further output.


d:\anaconda\envs\my-idaes-env\lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


# 4. Build and Run IDAES Flowsheet

This step builds an IDAES flowsheet and imports the surrogate model objects. As shown in the prior three examples, a single model object accounts for all input and output variables, and the JSON model serialized earlier may be imported into a single SurrogateBlock() component. While the serialization method and file structure differs slightly between the ALAMO, PySMO and Keras Python Wrappers, the three are imported similarly into IDAES flowsheets as shown below.

## 4.1 Build IDAES Flowsheet

This method builds an instance of the IDAES flowsheet model and solves the flowsheet using IPOPT. The method allows users to select a case and the surrogate model type to be used (i.e., alamo, pysmo, keras). The case argument consists of a list with values for the input variables (in this order, bypass split fraction and natural gas to steam ratio). Then the method fixes the input variables values to solve a square problem with IPOPT. 

In [7]:
# Import IDAES and Pyomo libraries
from pyomo.environ import ConcreteModel, SolverFactory, value, Var, Constraint, Set
from idaes.core.surrogate.surrogate_block import SurrogateBlock
from idaes.core.surrogate.alamopy import AlamoSurrogate
from idaes.core.surrogate.pysmo_surrogate import PysmoSurrogate
from idaes.core.surrogate.keras_surrogate import KerasSurrogate
from idaes.core import FlowsheetBlock


def build_flowsheet(case, surrogate_type: SurrType = None):
    print(case, " ", surrogate_type.value)
    # create the IDAES model and flowsheet
    m = ConcreteModel()
    m.fs = FlowsheetBlock(dynamic=False)

    # create flowsheet input variables
    m.fs.bypass_frac = Var(
        initialize=0.80, bounds=[0.1, 0.8], doc="natural gas bypass fraction"
    )
    m.fs.ng_steam_ratio = Var(
        initialize=0.80, bounds=[0.8, 1.2], doc="natural gas to steam ratio"
    )

    # create flowsheet output variables
    m.fs.steam_flowrate = Var(initialize=0.2, doc="steam flowrate")
    m.fs.reformer_duty = Var(initialize=10000, doc="reformer heat duty")
    m.fs.AR = Var(initialize=0, doc="AR fraction")
    m.fs.C2H6 = Var(initialize=0, doc="C2H6 fraction")
    m.fs.C3H8 = Var(initialize=0, doc="C3H8 fraction")
    m.fs.C4H10 = Var(initialize=0, doc="C4H10 fraction")
    m.fs.CH4 = Var(initialize=0, doc="CH4 fraction")
    m.fs.CO = Var(initialize=0, doc="CO fraction")
    m.fs.CO2 = Var(initialize=0, doc="CO2 fraction")
    m.fs.H2 = Var(initialize=0, doc="H2 fraction")
    m.fs.H2O = Var(initialize=0, doc="H2O fraction")
    m.fs.N2 = Var(initialize=0, doc="N2 fraction")
    m.fs.O2 = Var(initialize=0, doc="O2 fraction")

    # create input and output variable object lists for flowsheet
    inputs = [m.fs.bypass_frac, m.fs.ng_steam_ratio]
    outputs = [
        m.fs.steam_flowrate,
        m.fs.reformer_duty,
        m.fs.AR,
        m.fs.C2H6,
        m.fs.C3H8,
        m.fs.C4H10,
        m.fs.CH4,
        m.fs.CO,
        m.fs.CO2,
        m.fs.H2,
        m.fs.H2O,
        m.fs.N2,
        m.fs.O2,
    ]

    # create the Pyomo/IDAES block that corresponds to the surrogate
    # call correct PySMO object to use below (will let us avoid nested switches)

    # capture long output from loading surrogates (don't need to print it)
    stream = StringIO()
    oldstdout = sys.stdout
    sys.stdout = stream

    if surrogate_type == SurrType.ALAMO:
        surrogate = AlamoSurrogate.load_from_file("alamo_surrogate.json")
        m.fs.surrogate = SurrogateBlock()
        m.fs.surrogate.build_model(surrogate, input_vars=inputs, output_vars=outputs)
    elif surrogate_type == SurrType.NDCT:
        surrogate = AlamoSurrogate.load_from_file("DFCS_full.json")
        m.fs.surrogate = SurrogateBlock()
        m.fs.surrogate.build_model(surrogate, input_vars=inputs, output_vars=outputs)
    elif surrogate_type == SurrType.KERAS:
        keras_surrogate = KerasSurrogate.load_from_folder(
            keras_folder_name="keras_surrogate", keras_model_name="keras_model"
        )
        m.fs.surrogate = SurrogateBlock()
        m.fs.surrogate.build_model(
            keras_surrogate,
            formulation=KerasSurrogate.Formulation.FULL_SPACE,
            input_vars=inputs,
            output_vars=outputs,
        )
    elif SurrType.is_pysmo(
        surrogate_type
    ):  # surrogate is one of the three pysmo basis options
        surrogate = PysmoSurrogate.load_from_file(
            surrogate_type.value + "_surrogate.json"
        )
        m.fs.surrogate = SurrogateBlock()
        m.fs.surrogate.build_model(surrogate, input_vars=inputs, output_vars=outputs)
    else:
        raise ValueError(f"Unknown surrogate type: {surrogate_type}")

    # revert to standard output
    sys.stdout = oldstdout

    # fix input values and solve flowsheet
    m.fs.bypass_frac.fix(case[0])
    m.fs.ng_steam_ratio.fix(case[1])

    solver = SolverFactory("ipopt")
    try:  # attempt to solve problem
        results = solver.solve(m, tee=True)
    except:  # retry solving one more time
        results = solver.solve(m, tee=True)

    return (
        value(m.fs.steam_flowrate),
        value(m.fs.reformer_duty),
        value(m.fs.C2H6),
        value(m.fs.CH4),
        value(m.fs.H2),
        value(m.fs.O2),
    )

## 4.2 Model Size/Form Comparison

As mentioned above, as part of best practices the IDAES ML/AI demonstration includes the analysis of model/solver statistics and performance to determine the best surrogate model, including model size, model form, model trainer, etc. This section provides the rigorous analysis of solver performance comparing different surrogate models (ALAMO, PySMO polynomial, PysMO RBF, and PySMO Kriging).

To obtain the results, we run the flowsheet for ten different simulation cases for each surrogate model type. Since the simulation cases are obtained from the training data set we can compare model performance (absolute error of measurement vs predicted output values).

In [ ]:
import numpy as np
import pandas as pd


np.set_printoptions(precision=6, suppress=True)
csv_data = pd.read_csv(r"reformer-data.csv")  # 2800 data points
# extracting 10 data points out of 2800 data points, randomly selecting 10 cases to run
case_data = csv_data.sample(n=10)

# selecting columns that correspond to Input Variables
inputs = np.array(case_data.iloc[:, :2])

# selecting columns that correspond to Output Variables
cols = ["Steam_Flow", "Reformer_Duty", "C2H6", "CH4", "H2", "O2"]
outputs = np.array(case_data[cols])

case_data

,Bypass_Fraction,NG_Steam_Ratio,Steam_Flow,Reformer_Duty,AR,C2H6,C3H8,C4H10,CH4,CO,CO2,H2,H2O,N2,O2
2756,0.120290,1.168421,1.245639,39114.99976,0.004064,0.000632,0.000138,0.000079,0.019619,0.102605,0.054162,0.329404,0.152827,0.336469,3.620000e-20
2442,0.272464,1.021053,0.900235,30144.78779,0.004010,0.001709,0.000374,0.000214,0.052450,0.105092,0.049860,0.320895,0.132863,0.332534,2.780000e-20
1348,0.120290,0.884211,0.942645,33453.52312,0.004247,0.000661,0.000145,0.000083,0.020662,0.115367,0.048310,0.335487,0.123398,0.351641,2.270000e-20
1749,0.627536,1.094737,0.494136,17145.02348,0.003325,0.006375,0.001395,0.000797,0.194383,0.085521,0.044551,0.267845,0.117579,0.278230,2.620000e-20
1788,0.607246,1.084211,0.516044,17828.79525,0.003387,0.005960,0.001304,0.000745,0.181759,0.087359,0.044982,0.272620,0.118703,0.283181,2.620000e-20
2616,0.191304,1.168421,1.145085,36058.59104,0.004004,0.001078,0.000236,0.000135,0.033171,0.101084,0.053501,0.324521,0.150562,0.331708,3.570000e-20
2473,0.262319,1.136842,1.016300,32479.27439,0.003952,0.001599,0.000350,0.000200,0.049057,0.100582,0.052179,0.319525,0.144823,0.327731,3.340000e-20
582,0.505797,0.821053,0.491733,18935.37043,0.003760,0.004379,0.000958,0.000547,0.133813,0.103866,0.042170,0.295075,0.102252,0.313181,1.770000e-20
1884,0.556522,1.042105,0.560063,19373.72838,0.003536,0.005049,0.001105,0.000631,0.154063,0.092165,0.045581,0.283490,0.119395,0.294985,2.540000e-20
2454,0.272464,1.147368,1.011604,32227.05524,0.003935,0.001677,0.000367,0.000210,0.051418,0.099885,0.052250,0.318431,0.145463,0.326363,3.390000e-20


In [ ]:
# Import Auto-reformer training data
import numpy as np
import pandas as pd
import time
# 存储时间结果
time_costs = {}
np.set_printoptions(precision=6, suppress=True)
csv_data = pd.read_csv(r"reformer-data.csv")  # 2800 data points
# extracting 10 data points out of 2800 data points, randomly selecting 10 cases to run
case_data = csv_data.sample(n=20)

# selecting columns that correspond to Input Variables
inputs = np.array(case_data.iloc[:, :2])

# selecting columns that correspond to Output Variables
cols = ["Steam_Flow", "Reformer_Duty", "C2H6", "CH4", "H2", "O2"]
outputs = np.array(case_data[cols])

# For results comparison with minimum memory usage we will extract the values to plot on each pass
# note that the entire model could be returned and saved on each loop if desired

# create empty dictionaries so we may easily index results as we save them
# for convenience while plotting, each output variable has its own dictionary
# indexed by (case number, trainer type)
# trainers = ["alamo", "pysmo_poly", "pysmo_rbf", "pysmo_krig", "keras"]
# temporarily remove keras
trainers = [
    SurrType.ALAMO,
    SurrType.NDCT,
    SurrType.PYSMO_PLY,
    SurrType.PYSMO_RBF,
    SurrType.PYSMO_KRG,
    SurrType.KERAS,
]
cases = range(len(inputs))
steam_flow_error = {}
reformer_duty_error = {}
conc_C2H6 = {}
conc_CH4 = {}
conc_H2 = {}
conc_O2 = {}

i = 0
for case in inputs:  # each case is a value pair (bypass_frac, ng_steam_ratio)
    i += 1
    for trainer in trainers:
        start = time.time()
        [sf, rd, eth, meth, hyd, oxy] = build_flowsheet(case, surrogate_type=trainer)
        end = time.time()
        elapsed = end - start

        # 记录耗时
        time_costs[(i, trainer)] = elapsed

        steam_flow_error[(i, trainer)] = abs(
            (sf - value(outputs[i - 1, 0])) / value(outputs[i - 1, 0])
        )
        reformer_duty_error[(i, trainer)] = abs(
            (rd - value(outputs[i - 1, 1])) / value(outputs[i - 1, 1])
        )
        conc_C2H6[(i, trainer)] = abs(eth - value(outputs[i - 1, 2]))
        conc_CH4[(i, trainer)] = abs(meth - value(outputs[i - 1, 3]))
        conc_H2[(i, trainer)] = abs(hyd - value(outputs[i - 1, 4]))
        conc_O2[(i, trainer)] = abs(oxy - value(outputs[i - 1, 5]))




[0.252174 0.884211]   alamo
Ipopt 3.13.2: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale scientific
    

We can visualize these results by plotting a graph for each of the quantities above, creating a data series for each surrogate trainer. Some data series may overlay if values are identical for all cases:

In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler


records = []
for (case, trainer) in steam_flow_error.keys():
    records.append({
        "Case": case,
        "Trainer": str(trainer),
        "Steam_Flow_Error": steam_flow_error[(case, trainer)],
        "Reformer_Duty_Error": reformer_duty_error[(case, trainer)],
        "C2H6_Error": conc_C2H6[(case, trainer)],
        "CH4_Error": conc_CH4[(case, trainer)],
        "H2_Error": conc_H2[(case, trainer)],
        "O2_Error": conc_O2[(case, trainer)],
        "Time_Cost(s)": time_costs[(case, trainer)]
    })

results_df = pd.DataFrame(records)
print(f"📊 共收集 {len(results_df)} 条记录")


results_df["Trainer"] = results_df["Trainer"].astype(str)
results_df["Trainer"] = (
    results_df["Trainer"]
    .str.upper()
    .replace({
        "SURRTYPE.ALAMO": "ALAMO",
        "SURRTYPE.NDCT": "NDCT",
        "SURRTYPE.PYSMO_PLY": "PLY",
        "SURRTYPE.PYSMO_RBF": "RBF",
        "SURRTYPE.PYSMO_KRG": "KRG",
        "SURRTYPE.KERAS": "KERAS",
    })
)


trainer_order = ["ALAMO", "NDCT", "PLY", "RBF", "KRG", "KERAS"]
results_df["Trainer"] = pd.Categorical(results_df["Trainer"], categories=trainer_order, ordered=True)
results_df = results_df.sort_values(by=["Trainer", "Case"]).reset_index(drop=True)


results_df.to_csv("training_results.csv", index=False, encoding="utf-8-sig")
print("✅ 已保存: training_results.csv")


mean_df = results_df.groupby("Trainer", as_index=False).mean(numeric_only=True)
mean_df["Trainer"] = pd.Categorical(mean_df["Trainer"], categories=trainer_order, ordered=True)
mean_df = mean_df.sort_values("Trainer").reset_index(drop=True)
mean_df.to_csv("training_results_mean.csv", index=False, encoding="utf-8-sig")
print("✅ 已保存: training_results_mean.csv")

mean_df = pd.read_csv("training_results_mean.csv")


cols = [
    "Steam_Flow_Error",
    "Reformer_Duty_Error",
    "C2H6_Error",
    "CH4_Error",
    "H2_Error",
    "O2_Error",
    "Time_Cost(s)"
]


eps = 1e-12
log_vals = np.log10(mean_df[cols] + eps)


scaler = MinMaxScaler()
norm_vals = scaler.fit_transform(log_vals)
scores = 1 - norm_vals 


score_df = pd.DataFrame(scores, columns=[c + "_Score" for c in cols])
final_df = pd.concat([mean_df[["Trainer"]], score_df], axis=1)

print(final_df)


📊 共收集 120 条记录
✅ 已保存: training_results.csv
✅ 已保存: training_results_mean.csv
✅ 已保存: training_results_score_log_normalized.csv

📈 最终归一化得分表：
  Trainer  Steam_Flow_Error_Score  Reformer_Duty_Error_Score  \
0   ALAMO                0.997010                   0.719225   
1    NDCT                0.036742                   0.185954   
2     PLY                1.000000                   0.701747   
3     RBF                0.234164                   0.571503   
4     KRG                0.296354                   1.000000   
5   KERAS                0.000000                   0.000000   

   C2H6_Error_Score  CH4_Error_Score  H2_Error_Score  O2_Error_Score  \
0          0.220484     1.897306e-01    2.202236e-01        0.999961   
1          0.036018     2.220446e-16    1.000000e+00        1.000000   
2          0.657793     6.420567e-01    5.189618e-01        0.000000   
3          0.314708     2.862614e-01    3.741083e-01        1.000000   
4          1.000000     1.000000e+00    4.440892e-16  

C:\Users\16941\AppData\Local\Temp\ipykernel_29920\3224121727.py:56: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  mean_df = results_df.groupby("Trainer", as_index=False).mean(numeric_only=True)


In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import re
import os


mpl.rcParams['font.family'] = 'serif'
mpl.rcParams['font.serif'] = ['Times New Roman']
mpl.rcParams['font.weight'] = 'bold'


def plot_prediction(ax, cases, data_dict, trainers, ylabel, title, tag, yscale="log"):
    colors = {
    "ALAMO": "#DB443C",  
    "NDCT":  "#F57B56",  
    "PLY":   "#FDB670",  
    "RBF":   "#82B5D5",  
    "KRG":   "#5682BB", 
    "Keras": "#454A9F",  
}

    def simplify_name(name):
        name = str(name).upper()
        if "ALAMO" in name: return "ALAMO"
        if "NDCT" in name: return "NDCT"   
        if "PLY" in name:  return "PLY"
        if "RBF" in name:  return "RBF"
        if "KRG" in name:  return "KRG"  
        if "KERAS" in name: return "Keras"
        return name

    for trainer in trainers:
        tname = simplify_name(trainer)
        vals = [data_dict[(i, j)] for (i, j) in data_dict if j == trainer]

        ax.plot(
            cases, vals,
            lw=2.5 if tname == "NDCT" else 2,
            color=colors.get(tname, "black"),
            alpha=0.9,
            label=tname,
            zorder=1
        )
        ax.scatter(
            cases, vals,
            s=45,
            facecolors="white",
            edgecolors=colors.get(tname, "black"),
            linewidths=1,
            zorder=3
        )

    ax.set_xlabel("Cases", fontsize=14, fontweight="bold")
    ax.set_ylabel(ylabel, fontsize=16, fontweight="bold")
    ax.set_yscale(yscale)

    ax.tick_params(axis="both", direction="in", which="both", labelsize=14, width=1.5)
    for spine in ax.spines.values():
        spine.set_linewidth(1.5)

    ax.set_xticks(np.arange(2, max(cases)+1, 2))
    ax.set_yscale("log")
    #ax.set_ylim(1e-7+1e-8, 0.1-1e-4)    
    ax.tick_params(axis="y", which="minor", left=False) 
    

    #  (a) (b) ...
    ax.text(
        0.97, 0.97, tag,
        transform=ax.transAxes, fontsize=18, fontweight="bold",
        ha="right", va="top"
    )


    handles, labels = ax.get_legend_handles_labels()

    if "O2" in title:
        ax.legend(handles, labels, fontsize=13, frameon=False, loc="center right", bbox_to_anchor=(0.95, 0.67))
    else:
        ax.legend(handles, labels, fontsize=13, frameon=False, loc="best")


plots = [
    #  (steam_flow_error, "Absolute Error", "Steam Flow Prediction", "(b)"),
     #(reformer_duty_error, "Absolute Error", "Reformer Duty Prediction", "(a)"),
    # (conc_C2H6, "Absolute Error", "C2H6 Mole Fraction Prediction (O(1E-2))", "(a)"),
    # (conc_CH4, "Absolute Error", "CH4 Mole Fraction Prediction (O(1E-1))", "(b)"),
    #  (conc_H2, "Absolute Error", "H2 Mole Fraction Prediction (O(1E-1))", "(c)"),
    #  (conc_O2, "Absolute Error", "O2 Mole Fraction Prediction (O(1E-20))", "(d)"),
]


save_dir = r"D:\lib\gali\Manuscript\fig8-compare"
os.makedirs(save_dir, exist_ok=True)

saved_files = []
for data_dict, ylabel, title, tag in plots:
    fig, ax = plt.subplots(figsize=(7, 7))
    plot_prediction(ax, cases, data_dict, trainers, ylabel, title, tag)

    fname = re.sub(r'[^A-Za-z0-9]+', '_', title).strip('_').lower() + ".svg"
    full_path = os.path.join(save_dir, fname)


    fig.savefig(full_path, format="svg", bbox_inches="tight")

    plt.close(fig)
    saved_files.append(full_path)

for f in saved_files:
    print(" -", f)


## 4.3 Comparing Surrogate Optimization
Extending this analysis, we will run a single optimization scenario for each surrogate model and compare results. As in previous examples detailing workflows for [ALAMO](alamo_flowsheet_optimization_src_test.ipynb), [PySMO](pysmo_flowsheet_optimization_src_test.ipynb) and [Keras](keras_flowsheet_optimization_src_test.ipynb), we will optimize hydrogen production while restricting nitrogen below 34 mol% in the product stream.

In [9]:
# Import additional Pyomo libraries
from pyomo.environ import Objective, maximize


def run_optimization(surrogate_type=None):
    print(surrogate_type)
    # create the IDAES model and flowsheet
    m = ConcreteModel()
    m.fs = FlowsheetBlock(dynamic=False)

    # create flowsheet input variables
    m.fs.bypass_frac = Var(
        initialize=0.80, bounds=[0.1, 0.8], doc="natural gas bypass fraction"
    )
    m.fs.ng_steam_ratio = Var(
        initialize=0.80, bounds=[0.8, 1.2], doc="natural gas to steam ratio"
    )

    # create flowsheet output variables
    m.fs.steam_flowrate = Var(initialize=0.2, doc="steam flowrate")
    m.fs.reformer_duty = Var(initialize=10000, doc="reformer heat duty")
    m.fs.AR = Var(initialize=0, doc="AR fraction")
    m.fs.C2H6 = Var(initialize=0, doc="C2H6 fraction")
    m.fs.C3H8 = Var(initialize=0, doc="C3H8 fraction")
    m.fs.C4H10 = Var(initialize=0, doc="C4H10 fraction")
    m.fs.CH4 = Var(initialize=0, doc="CH4 fraction")
    m.fs.CO = Var(initialize=0, doc="CO fraction")
    m.fs.CO2 = Var(initialize=0, doc="CO2 fraction")
    m.fs.H2 = Var(initialize=0, doc="H2 fraction")
    m.fs.H2O = Var(initialize=0, doc="H2O fraction")
    m.fs.N2 = Var(initialize=0, doc="N2 fraction")
    m.fs.O2 = Var(initialize=0, doc="O2 fraction")

    # create input and output variable object lists for flowsheet
    inputs = [m.fs.bypass_frac, m.fs.ng_steam_ratio]
    outputs = [
        m.fs.steam_flowrate,
        m.fs.reformer_duty,
        m.fs.AR,
        m.fs.C2H6,
        m.fs.C3H8,
        m.fs.C4H10,
        m.fs.CH4,
        m.fs.CO,
        m.fs.CO2,
        m.fs.H2,
        m.fs.H2O,
        m.fs.N2,
        m.fs.O2,
    ]

    # create the Pyomo/IDAES block that corresponds to the surrogate
    # call correct PySMO object to use below (will let us avoid nested switches)

    # capture long output from loading surrogates (don't need to print it)
    stream = StringIO()
    oldstdout = sys.stdout
    sys.stdout = stream

    if surrogate_type == SurrType.ALAMO:
        surrogate = AlamoSurrogate.load_from_file("alamo_surrogate.json")
        m.fs.surrogate = SurrogateBlock()
        m.fs.surrogate.build_model(surrogate, input_vars=inputs, output_vars=outputs)
    elif surrogate_type == SurrType.NDCT:
        surrogate = AlamoSurrogate.load_from_file("DFCS_full.json")
        m.fs.surrogate = SurrogateBlock()
        m.fs.surrogate.build_model(surrogate, input_vars=inputs, output_vars=outputs)
    elif surrogate_type == SurrType.KERAS:
        keras_surrogate = KerasSurrogate.load_from_folder(
            keras_folder_name="keras_surrogate", keras_model_name="keras_model"
        )
        m.fs.surrogate = SurrogateBlock()
        m.fs.surrogate.build_model(
            keras_surrogate,
            formulation=KerasSurrogate.Formulation.FULL_SPACE,
            input_vars=inputs,
            output_vars=outputs,
        )
    elif SurrType.is_pysmo(
        surrogate_type
    ):  # surrogate is one of the three pysmo basis options
        surrogate = PysmoSurrogate.load_from_file(
            surrogate_type.value + "_surrogate.json"
        )
        m.fs.surrogate = SurrogateBlock()
        m.fs.surrogate.build_model(surrogate, input_vars=inputs, output_vars=outputs)
    else:
        raise ValueError(f"Unknown surrogate type: {surrogate_type}")

    # revert to standard output
    sys.stdout = oldstdout

    # unfix input values and add the objective/constraint to the model
    m.fs.bypass_frac.unfix()
    m.fs.ng_steam_ratio.unfix()
    m.fs.obj = Objective(expr=m.fs.H2, sense=maximize)
    m.fs.con = Constraint(expr=m.fs.N2 <= 0.34)

    solver = SolverFactory("ipopt")
    try:  # attempt to solve problem
        results = solver.solve(m, tee=True)
    except:  # retry solving one more time
        results = solver.solve(m, tee=True)

    return inputs, outputs

In [10]:
# create list objects to store data, run optimization
results = {}
for trainer in trainers:
    inputs, outputs = run_optimization(trainer)
    for var in inputs:
        results[(var.name, trainer)] = value(var)
    for var in outputs:
        results[(var.name, trainer)] = value(var)

SurrType.ALAMO
Ipopt 3.13.2: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale scientific
        computati

In [11]:
# print results as a table
df_index = []
for var in inputs:
    df_index.append(var.name)
for var in outputs:
    df_index.append(var.name)
df_cols = trainers

df = pd.DataFrame(index=df_index, columns=df_cols)
for i in df_index:
    for j in df_cols:
        df[j][i] = results[(i, j)]

df  # display results table

C:\Users\16941\AppData\Local\Temp\ipykernel_160684\4045444663.py:12: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df[j][i] = results[(i, j)]


,SurrType.ALAMO,SurrType.NDCT,SurrType.PYSMO_PLY,SurrType.PYSMO_RBF,SurrType.PYSMO_KRG,SurrType.KERAS
fs.bypass_frac,0.1,0.100046,0.1,0.1,0.1,0.1
fs.ng_steam_ratio,1.130836,1.115732,1.126451,1.124305,1.124456,1.100677
fs.steam_flowrate,1.233375,1.187418,1.228592,1.226153,1.226396,1.187883
fs.reformer_duty,39226.32453,38005.261553,39131.26643,39062.177089,39081.12203,38002.377259
fs.AR,0.004107,0.004107,0.004107,0.004107,0.004107,0.004117
fs.C2H6,0.000497,0.000569,0.000519,0.000545,0.000518,0.000491
fs.C3H8,0.000109,0.000125,0.000114,0.000119,0.000114,0.000125
fs.C4H10,0.000062,0.000071,0.000065,0.000068,0.000065,0.000069
fs.CH4,0.015527,0.017723,0.0162,0.016991,0.016222,0.017458
fs.CO,0.104687,0.10511,0.104419,0.104856,0.104844,0.106085


In [ ]:
df_index = []
for var in inputs:
    df_index.append(var.name)
for var in outputs:
    df_index.append(var.name)
df_cols = trainers

df = pd.DataFrame(index=df_index, columns=df_cols)
for i in df_index:
    for j in df_cols:
        df.loc[i, j] = results[(i, j)]

df  

df.to_csv("optimization_results.csv", encoding="utf-8-sig") 
